In [87]:
from datetime import date,datetime
import json
import csv
import time
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass


@dataclass
class Holiday:
    name: str
    date: datetime
    tag: str

    def __str__ (self):
        return self.name + " (" + self.date.strftime("%Y-%m-%d") + ")"

@dataclass
class HolidayList:
    holidays: list

    def addHoliday(holidayObj):
        pass

    def removeHoliday(HolidayName, Date):
        pass

    def save_to_json(filelocation):
        pass

    def read_json(self,filelocation):
        
        try:
            with open(filelocation,"r") as file:
                load = json.load(file)
                
                for x in load["holidays"]:
                    self.holidays.append(Holiday(x["name"],datetime.strptime(x["date"], "%b %d %Y"),x["tag"]))
        except:
            print("404")

    def save_to_csv(filelocation):
        pass

    def tag_search():
        pass

    def date_search():
        pass

    def numHolidays(self):
        return len(self.holidays)

    def scrapeHolidays(self):
        
        todays_date = date.today()

        counter = todays_date.year - 2

        while counter < todays_date.year + 3:

            fetchedData = requests.get("https://www.timeanddate.com/holidays/us/"+ str(counter) +"?hol=9565233").text

            soup = BeautifulSoup(fetchedData, "lxml")

            data = soup.find_all("tr", class_= "showrow")

            for holiday in data:

                name = holiday.find("a").text
                tag = holiday.find_all("td")[2].get_text()
                holiday_date = datetime.strptime(holiday.find("th").text + " " +str(counter), "%b %d %Y")

                self.holidays.append(Holiday(name,holiday_date.date(),tag))
            
            counter += 1

def main():

    holiday_list = HolidayList([])
    holiday_list.read_json("Data/holidays.json")

    while(True):
        
        print("Holiday Menu")
        print("================")
        print("1. Add a Holiday")
        print("2. Remove a Holiday")
        print("3. Save Holiday List")
        print("4. View Holidays")
        print("5. Tag Search")
        print("6. Date Search")
        print("7. Exit\n")
        break

if __name__ == "__main__":
    main()


Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Tag Search
6. Date Search
7. Exit

